## OpeanAI Proxy

You may have access to a large langugage model through a HTTP Proxy like this one
https://github.com/derickson/ExpressOpenAIChatProxy

Assuming you know a temporary key for accessing ChatGPT 3.5 through this proxy 
similar to the Elastic GenAI workshop series here is how to set that up


In [ ]:
! pip install -q openai==1.14.3 python_dotenv
import os
import openai
from dotenv import load_dotenv

## suppress some warnings
import warnings, os
warnings.filterwarnings("ignore", category=DeprecationWarning, message="on_submit is deprecated.*")

FILE="Studygroup"

# workshop environment - this is where you'll enter a key
! pip install -qqq git+https://github.com/elastic/notebook-workshop-loader.git@main
from notebookworkshoploader import loader
from dotenv import load_dotenv
loader.load_remote_env(file=FILE, env_url="https://notebook-workshop-api-voldmqr2bq-uc.a.run.app")

In [ ]:
import os, secrets, requests, json
import openai
from openai import OpenAI
from requests.auth import HTTPBasicAuth

#if using the Elastic AI proxy, then generate the correct API key
if os.environ['ELASTIC_PROXY'] == "True":

    if "OPENAI_API_TYPE" in os.environ: del os.environ["OPENAI_API_TYPE"]

    #generate and share "your" unique hash
    os.environ['USER_HASH'] = secrets.token_hex(nbytes=6)
    print(f"Your unique user hash is: {os.environ['USER_HASH']}")

    #get the current API key and combine with your hash
    os.environ['OPENAI_API_KEY'] = f"{os.environ['OPENAI_API_KEY']} {os.environ['USER_HASH']}"
else:
    openai.api_type = os.environ['OPENAI_API_TYPE']
    openai.api_version = os.environ['OPENAI_API_VERSION']

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.default_model = os.environ['OPENAI_API_ENGINE']

import json
# pretty printing JSON objects
def json_pretty(input_object):
  print(json.dumps(input_object, indent=4))


import textwrap
# wrap text when printing, because colab scrolls output to the right too much
def wrap_text(text, width):
    wrapped_text = textwrap.wrap(text, width)
    return '\n'.join(wrapped_text)

def print_light_blue(text):
    print(f'\033[94m{text}\033[0m')

import ipywidgets as widgets
from IPython.display import display

class NotebookChatExperience:
    def __init__(self, ai_response_function, ai_name = "AI"):
        self.ai_name = ai_name
        self.ai_response_function = ai_response_function
        self.chat_history = widgets.Textarea(
            value='',
            placeholder='Chat history will appear here...',
            description='Chat:',
            disabled=True,
            layout=widgets.Layout(width='700px', height='300px')  # Adjust the size as needed
        )
        self.user_input = widgets.Text(
            value='',
            placeholder='Type your message here...',
            description='You:',
            disabled=False,
            layout=widgets.Layout(width='700px')  # Adjust the size as needed
        )
        self.user_input.on_submit(self.on_submit)
        display(self.chat_history, self.user_input)

    def on_submit(self, event):
        user_message = self.user_input.value
        ai_name = self.ai_name
        self.chat_history.value += f"\nYou: {user_message}"
        ai_message = self.ai_response_function(user_message)
        self.chat_history.value += f"\n{ai_name}: {ai_message}"
        self.user_input.value = ''  # Clear input for next message

    def clear_chat(self):
        self.chat_history.value = ''  # Clear the chat history

## ********** Example usage:

## ********** Define a simple AI response function
# def simple_ai_response(user_message):
    # return f"AI > Echo: {user_message}"

## ********** Create an instance of the chat interface
#chat_instance = NotebookChatExperience(simple_ai_response)

In [ ]:
## openai will look through environment variables itself to find the the OPENAI_API_KEY 

from openai import OpenAI
client = OpenAI(api_key=openai.api_key, base_url=openai.api_base)
models_list = client.models.list().dict()

# Convert the dictionary to a pretty-printed JSON string
pretty_json = json.dumps(models_list, indent=4)

print(pretty_json)

## an error here is expected. the proxy may not implement all options the OpenAI does

In [ ]:
# Call the OpenAI ChatCompletion API
def chatCompletion(messages, max_tokens=100):
    client = OpenAI(api_key=openai.api_key, base_url=openai.api_base)
    completion = client.chat.completions.create(
        model=openai.default_model,
        max_tokens=max_tokens,
        messages=messages
    )
    return completion

prompt="Hello, is ChatGPT online and working?"

messages = [{"role": "user", "content": prompt}]

completion = chatCompletion(messages)

response_text = completion.choices[0].message.content

print(wrap_text(completion.model_dump_json(),70))

print("\n", wrap_text(response_text,70))

In [ ]:
client = OpenAI(api_key=openai.api_key, base_url=openai.api_base)
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What color is the sky?"}
  ],
  max_tokens=100
)


print(wrap_text(completion.choices[0].message.content,70))

In [ ]:
retrieved_context = "The color of the sky is purple today"


system_prompt = f"""
You are a helpful AI assistant that answers questions using the provided context.
If you can't answer the question using the provided context, say "I don't know".
Don't apologize, and don't say you are an AI assistant.

Context: {retrieved_context}
"""

question = "What is the sky like today?"

messages = [
    {"role":"system", "content": system_prompt},
    {"role":"user", "content": question}
]


completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  max_tokens=100,
  temperature=0,
)


print(wrap_text(completion.choices[0].message.content,70))

In [ ]:
def openai_ai_response(user_message):
  messages = [{"role": "user", "content": user_message}]
  completion = chatCompletion(messages)
  response_text = completion.choices[0].message.content
  return response_text

chat_instance = NotebookChatExperience(openai_ai_response)